In [1]:
from statsmodels.datasets import grunfeld

data = grunfeld.load_pandas().data
data.year = data.year.astype(int)
data = data.set_index(['firm', 'year'])
data.head()

invest   value  capital
firm           year                         
General Motors 1935   317.6  3078.5      2.8
               1936   391.8  4661.7     52.6
               1937   410.6  5387.1    156.9
               1938   257.7  2792.2    209.2
               1939   330.8  4313.2    203.4

We will use the data set studied in T.A. Mroz (1987), "The Sensitivity of an Empirical Model of Married Women's
Hours of Work to Economic and Statistical Assumptions," Econometrica 55,
765-799.  This data set is used for examples in Wooldridge's Introduction to Econometrics and also in Sheppard's description of his linearmodels library.

In [ ]:
from linearmodels.datasets import mroz
data = mroz.load()
data = data.dropna()
print(mroz.DESCR)
data.head()

In [6]:
from linearmodels import PanelOLS

result = PanelOLS.from_formula("invest ~ value + capital + EntityEffects", data=data).fit()
result.summary

Dep. Variable:,invest,R-squared:,0.7667
Estimator:,PanelOLS,R-squared (Between):,0.8223
No. Observations:,220,R-squared (Within):,0.7667
Date:,"Sun, Aug 21 2022",R-squared (Overall):,0.8132
Time:,19:17:52,Log-likelihood,-1167.4
Cov. Estimator:,Unadjusted,,
,,F-statistic:,340.08
Entities:,11,P-value,0.0000
Avg Obs:,20.000,Distribution:,"F(2,207)"
Min Obs:,20.000,,
Max Obs:,20.000,F-statistic (robust):,340.08
